In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import csv
import torch
from transformers import BertTokenizer, BertModel
import torch.nn as nn
import re
import pandas as pd

# Initialize the BERTSUM model and tokenizer
class BERTSUM(nn.Module):
    def __init__(self, model_name='bert-base-uncased'):
        super(BERTSUM, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.linear = nn.Linear(self.bert.config.hidden_size, 1)
        self.softmax = nn.Softmax(dim=1)  # Per word softmax for importance

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        word_output = outputs.last_hidden_state  # Word token representations
        scores = self.linear(word_output)
        scores = self.softmax(scores)
        return scores.squeeze(-1)

# Load pre-trained model and tokenizer
model = BERTSUM()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to extract most important sentences based on word importance scores
def summarize_text(text, model, tokenizer, num_sentences=1):
    paragraphs = text.split('\n')
    paragraphs = [p for p in paragraphs if p.strip()]  # Hapus paragraf kosong
    important_sentences = []
    
    for paragraph in paragraphs:
        sentences = re.split(r'(?<=[.!?]) +', paragraph)  # Pisah paragraf menjadi kalimat
        paragraph_summary = []

        for sentence in sentences:
            inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
            
            # Hapus token_type_ids jika ada
            if 'token_type_ids' in inputs:
                del inputs['token_type_ids']
            
            tokenized_words = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
            
            with torch.no_grad():
                scores = model(**inputs)
            
            # Cek jika scores tidak kosong
            if scores.numel() > 0:  # Memastikan tensor tidak kosong
                k = min(5, scores.size(0))  # Set k tidak lebih besar dari ukuran tensor
                top_word_indices = torch.topk(scores.squeeze(), k=k).indices
                top_words = [tokenized_words[i.item()] for i in top_word_indices if tokenized_words[i.item()] not in tokenizer.all_special_tokens]

                # Jika kalimat memiliki kata-kata penting, tambahkan ke ringkasan paragraf
                if len(top_words) > 0:
                    paragraph_summary.append(sentence)  # Menambahkan kalimat ke ringkasan paragraf

            else:
                print("Scores tensor is empty. Skipping this sentence.")

        # Kombinasikan kalimat paling penting dari setiap paragraf
        if paragraph_summary:
            important_sentences.append(paragraph_summary[0])  # Ambil kalimat paling penting dari setiap paragraf

    # Gabungkan kalimat yang dipilih menjadi ringkasan singkat
    summary = ' '.join(important_sentences)
    return summary

# Function to scrape website and summarize
def scrape_website(url, title_tag, title_class, date_tag, date_class, content_tag, content_class):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    title = soup.find(title_tag, class_=title_class).get_text(strip=True)
    date = soup.find(date_tag, class_=date_class).get_text(strip=True)
    content = soup.find(content_tag, class_=content_class).get_text(strip=True)

    # Automatically summarize the content using BERTSUM
    summary = summarize_text(content, model, tokenizer, num_sentences=1)  # Adjust to only 1 sentence per paragraph

    return {'Title': title, 'Date': date, 'Summary': summary}

# URL and HTML tag details for the website
websites = [
    {
        'url': 'https://blockchainmedia.id/michael-saylor-tetap-optimis-meski-harga-btc-macet/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/masa-depan-toncoin-harga-ton-di-ambang-kebangkitan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/think-tank-korea-selatan-ragukan-etf-bitcoin-ada-apa/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dogwifhat-wif-keluar-dari-50-besar-harga-anjlok-38-persen/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/xrp-ripple-terus-berkonsolidasi-kapan-harga-akan-naik/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/minat-pada-shiba-inu-meningkat-biaya-shibarium-meroket/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/prospek-titik-balik-bch-prediksi-bullish-toncoin-ton-hingga-inovasi-platform-blockdag/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/bagaimana-pertumbuhan-ethereum-eth-dan-tren-pasar-bitcoin-btc-picu-peningkatan-presale-furrever-token/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/pasar-etf-bitcoin-spot-masih-merah-sol-meluncur-di-bawah-ma200-dan-rollblock-rblk-coba-menarik-perhatian/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/uniswap-dan-etc-alami-beda-nasib-influencer-ulas-teknologi-blockdag/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/mt-gox-akan-mulai-bayar-bitcoin-juli-2024-kreditur-bersorak/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-doge-dalam-zona-beli-kesempatan-emas-bagi-investor/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/volume-toncoin-naik-drastis-apa-artinya-bagi-pasar/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/scaramucci-biden-menang-harga-bitcoin-bisa-capai-us250-000/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analisis-harga-pepe-shib-dan-prospek-baru-rollblock-rblk/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ketokohan-gary-gensler-di-balik-penerbitan-meme-coin-sec-berbasis-blockchain-ripple-xrp/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/5-crypto-mining-pilihan-blockdag-hingga-zcash/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analis-popular-bitcoin-akan-melejit-ke-us70-000-dalam-waktu-dekat/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/sec-gensler-etf-ethereum-berjalan-lancar-dan-mendapat-dukungan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/solana-hadapi-rival-baru-potensi-reli-sui-hingga-teknologi-canggih-blockdag/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-wif-lesu-pasca-akumulasi-whale-bagaimana-prediksinya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/willy-woo-bitcoin-akan-menyaingi-dolar-as-di-masa-depan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/whale-dogecoin-serok-serok-di-tengah-pasar-yang-lesu-tanda-apakah-ini/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/altseason-altcoin-siap-menggebrak-meski-bitcoin-anjlok/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/bear-di-bitcoin-masih-pegang-kendali-perdagangan-etf-ethereum-spot-diprediksi-juli-dan-rollblock-rblk-masuki-kripto-berbasis-permainan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/monetisasi-data-perjalanan-dan-token-ptrn-dari-pathearn/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ekspansi-ripple-stablecoin-baru-dan-peluang-etf-xrp/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ringkasan-berita-kripto-hari-ini-setelah-burn-not-us3-juta-vaneck-ajukan-proposal-etf-sol-potensi-bullish-xrp-serta-ulasan-rblk-di-sektor-permainan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ringkasan-berita-kripto-hari-ini-setelah-burn-not-us3-juta-vaneck-ajukan-proposal-etf-sol-potensi-bullish-xrp-serta-ulasan-rblk-di-sektor-permainan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/vaneck-ajukan-etf-solana-spot-pertama-dunia-kripto-berguncang-analis-prediksi-meluncur-tahun-2025/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/token-notcoin-not-melejit-usai-burn-apa-selanjutnya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/siapa-susul-proposal-etf-solana-spot-prediksi-gerakan-harga-btc-berikutnya-dan-jejak-awal-rollblock-rblk-di-industri-hiburan-permainan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analisis-pertumbuhan-toncoin-ton-render-rndr-hingga-blockdag/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ulasan-terbaru-inovasi-dan-pertumbuhan-blockdag/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/1-juta-alamat-kini-memegang-1-bitcoin-btc-meski-pasar-bergejolak/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-notcoin-uji-level-kritis-pemulihan-di-depan-mata/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/etf-ethereum-bisa-dorong-harga-eth-melesat-ke-us6-500-ini-alasannya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/kembali-berulah-whale-pindahkan-155-triliun-shib-untuk-apa/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/berita-kripto-terkini-rumor-penundaan-persetujuan-perdagangan-etf-ethereum-usai-harga-tia-luruh-46-persen-dan-rollblock-cicipi-industri-permainan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/prediksi-harga-btc-terbaru-bisa-terjungkal-ke-us55-ribu-dan-gambaran-testnet-blockchain-blockdag/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analis-harga-xrp-bisa-melesat-ke-us070-ini-alasannya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/volume-cardano-ada-meroket-di-tengah-penurunan-pasar-kripto-ada-apa/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analisis-etf-solana-dan-potensi-lonjakan-harga-yang-menggiurkan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/kapitulasi-penambang-bitcoin-meningkat-sejak-halving-april-ada-apa/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analisis-teknikal-kripto-kaspa-kas-usai-cetak-ath-dan-rollblock-yang-berfitur-buy-back/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/bitcoin-anjlok-dua-kali-begini-analisis-pasarnya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-ethereum-eth-melonjak-siap-untuk-dorongan-bullish-lainnya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/investor-kembali-akumulasi-shib-harga-siap-pulih/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dogecoin-beri-tanda-apakah-harga-akan-segera-naik-lagi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/sony-akuisisi-crypto-exchange-pepe-menguat-lagi-hari-ini-dan-peran-rollblock-rblk-di-industri-hiburan-dan-permainan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/solana-bersiap-jadi-kripto-terbesar-ketiga-tahun-ini-ini-alasannya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/solana-bersiap-jadi-kripto-terbesar-ketiga-tahun-ini-ini-alasannya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/solana-bersiap-jadi-kripto-terbesar-ketiga-tahun-ini-ini-alasannya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/solana-bersiap-jadi-kripto-terbesar-ketiga-tahun-ini-ini-alasannya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/perbendaharaan-polkadot-dalam-kondisi-krisis-ini-faktanya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ripple-pindahkan-1-milyar-xrp-dari-dompet-escrow-untuk-apa/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ripple-pindahkan-1-milyar-xrp-dari-dompet-escrow-untuk-apa/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ripple-pindahkan-1-milyar-xrp-dari-dompet-escrow-untuk-apa/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analis-harga-doge-akan-mendaki-ke-us1-dalam-siklus-bull-terbaru/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/aktivitas-pengembangan-cardano-merosot-ada-apa/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/mengenal-dasia-platform-pemasaran-digital-berbasis-teknologi-ai-dan-blockchain/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ulasan-kripto-hari-ini-prediksi-xrp-us144-nasib-eth-berikutnya-dan-rollblock-bertenaga-ai/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ulasan-kripto-hari-ini-prediksi-xrp-us144-nasib-eth-berikutnya-dan-rollblock-bertenaga-ai/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/penyebab-bitcoin-turun-menghujam-us60-ribu-lagi-dan-gambaran-rollblock-rblk-yang-diperkuat-kecerdasan-buatan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analis-reku-ada-indikasi-strong-buy-untuk-altcoin/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/bitwise-etf-ethereum-bisa-debut-pertengahan-juli/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/sentimen-bearish-meningkat-harga-floki-terancam-turun-lagi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-xrp-digadang-siap-meroket-hingga-7-600-persen/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/robert-kiyosaki-prediksi-crash-besar-dan-kebangkitan-harga-btc-ke-us10-juta/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-btc-terjun-bebas-ke-us53-571-jauh-di-bawah-ema-200-dan-rollblock-tawarkan-permainan-beragam/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dogecoin-alami-penurunan-besar-inilah-penyebabnya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-xrp-terus-turun-kejelasan-hukum-jadi-pertanyaan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-xrp-terus-turun-kejelasan-hukum-jadi-pertanyaan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/whale-antusias-menyambut-fetch-ai-fet-berubah-jadi-asi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ada-antusiasme-baru-terhadap-etf-ethereum-di-pemilu-as-pertanda-baik/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-shib-rebound-15-persen-apakah-ini-awal-kenaikan-baru/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-eth-berpeluang-naik-dan-saingi-btc-etf-ethereum-segera-hadir/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-eth-berpeluang-naik-dan-saingi-btc-etf-ethereum-segera-hadir/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-eth-berpeluang-naik-dan-saingi-btc-etf-ethereum-segera-hadir/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/worldcoin-mencoba-pulih-pembalikan-harga-segera-terjadi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-shiba-inu-melonjak-burn-shib-melejit-781-persen/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/prediksi-baru-bitcoin-bisa-meroket-ke-us225-000-dalam-beberapa-bulan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-xrp-to-the-moon-spekulasi-meningkat-setelah-postingan-bitstamp/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/etf-solana-di-ambang-persetujuan-apa-dampaknya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/prediksi-harga-shiba-inu-shib-bisa-melesat-hingga-1-487-persen/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analis-harga-bitcoin-perlu-mencapai-us50-000-sebelum-kembali-dalam-lompatan-bullish/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/prediksi-harga-doge-bisakah-mempertahankan-momentumnya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/worldcoin-perkenalkan-world-chain-jaringan-layer-2-terbaru/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/etf-bitcoin-as-cetak-rekor-arus-masuk-di-tengah-pasar-lesu-blackrock-memimpin/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/pengguna-ripple-meningkat-volume-transaksi-sentuh-us1-milyar/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/burn-rate-shiba-inu-shib-melejit-347-persen-dalam-sehari/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/bitcoin-kesulitan-di-us60-000-apa-yang-terjadi-selanjutnya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/solana-siap-melejit-harga-sol-diprediksi-sentuh-us600/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dominasi-bitcoin-tinggi-altseason-masih-bisa-diharapkan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/aktivitas-shibarium-turun-bagaimana-nasib-harga-shib/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/etf-ethereum-diprediksi-menarik-us10-milyar-arus-masuk-eth-siap-meroket/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/korelasi-bitcoin-dan-sp-500-kian-renggang-apa-penyebabnya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/masa-depan-ripple-dapatkah-pemilu-as-mengubah-permainan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/gaming-token-superblok-sebagai-cara-baru-untuk-game-dan-peluang-menambah-pendapatan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/perkembangan-terkini-kripto-ripple-xrp-hijau-benderang-nasib-pepecoin-pepe-serta-presale-furrever-token-mendekati-tahap-akhir/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/varia-kabar-kripto-peluang-crossover-bullish-eth-potensi-doge-luruh-lagi-dan-furrever-token-segera-di-pancakeswap/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ajaib-kripto-bitcoin-masih-berpotensi-kembali-melemah-ke-us54-ribu/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/aset-kripto-di-tengah-anomali-pasca-rilis-data-cpi-dan-potensi-the-fed-pangkas-suku-bunga-di-september/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/etf-bitcoin-bangkit-arus-masuk-dana-meningkat-hebat/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/whale-litecoin-menggila-transaksi-us285-milyar-setiap-hari/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/cardano-naik-22-persen-dalam-seminggu-harga-ada-siap-melejit-lebih-tinggi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/worldcoin-siap-mempercepat-pembukaan-tokennya-apa-dampaknya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-xrp-naik-pesat-kepercayaan-investor-menguat-lagi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-xrp-naik-pesat-kepercayaan-investor-menguat-lagi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-toncoin-ton-menguat-lagi-investor-optimis-ath-baru/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-toncoin-ton-menguat-lagi-investor-optimis-ath-baru/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-eth-melonjak-akankah-melesat-lampaui-level-us3-500/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/doge-berjuang-naik-apa-peran-whale-dalam-pergerakan-ini/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/whale-bitcoin-serok-71-000-btc-di-tengah-penurunan-harga/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/sec-diprediksi-akan-setujui-etf-ethereum-minggu-ini/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dompet-whale-yang-tertidur-12-tahun-bangkit-dan-pindahkan-1-000-btc-pertanda-apa/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-xrp-naik-26-persen-spekulasi-penutupan-investigasi-sec-kian-memanas/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ekosistem-shiba-inu-menanjak-aktivitas-whale-jadi-sorotan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ai-meramal-harga-bitcoin-btc-pasca-pembayaran-mt-gox/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/nascent-serok-141-milyar-token-harga-pepe-coin-meroket/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/sambut-etf-ethereum-optimisme-investor-institusi-lebih-tinggi-daripada-ritel/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ada-tanda-bullish-analis-prediksi-harga-sol-tembus-us174/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/konsolidasi-harga-bonk-apakah-penembusan-akan-terjadi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ton-perkenalkan-teleport-btc-jembatan-bitcoin-baru/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/staking-ethereum-capai-rekor-tertinggi-jelang-etf-spot-tanda-bullish/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dogecoin-melampaui-90-juta-dompet-siap-melejit-ke-us1/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/strategi-baru-ripple-menghindari-ipo-fokus-tender-saham/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dogwifhat-wif-mendaki-kuat-kembali-ke-50-kripto-teratas/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/bitcoin-diproyeksi-melonjak-ke-us200-000-setelah-kapitulasi-penambang/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/polygon-siap-bertransformasi-migrasi-koin-matic-ke-pol/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ada-pola-bullish-pennant-sinyal-melesat-untuk-harga-sol/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/sistem-windows-crash-bitcoin-dan-blockchain-tetap-berfungsi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/whale-ethereum-serok-serok-harga-eth-siap-melonjak/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dogecoin-turun-lagi-whale-mulai-borong-doge-lagi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/prediksi-bullish-cardano-harga-ada-siap-menuju-us1/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/arus-masuk-etf-bitcoin-naik-institusi-kian-berminat/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-xrp-melonjak-harapan-baru-dari-pertemuan-sec/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/prediksi-para-ahli-shiba-inu-berpotensi-naik-ke-us0000045/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analis-senior-peter-brandt-dan-prediksi-bearish-pada-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/melesat-harga-polkadot-dot-berpotensi-naik-hingga-227-persen/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ada-transaksi-50-triliun-shiba-inu-shib-imbas-peretasan-wazirx/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/garlinghouse-pertempuran-ripple-vs-sec-akan-segera-berakhir/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analisis-harga-xrp-siap-untuk-kenaikan-besar-melawan-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/etf-ethereum-resmi-disetujui-sec-siap-diperdagangkan-hari-ini/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dogecoin-mendaki-tinggi-ada-potensi-kenaikan-harga-lagi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ibit-blackrock-menerima-dana-masuk-us523-juta-bitcoin-melonjak/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/tren-kripto-terkini-momentum-filecoin-dan-tron-bagaimana-dengan-blockdag/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/prediksi-harga-blockdag-hingga-dinamika-pasar-celestia-dan-algorand/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/cfx-perkuat-komitmen-pengembangan-pasar-kripto-indonesia-ini-respons-bappebti/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/perdagangan-etf-spot-ethereum-dimulai-hari-ini-berikut-potensinya-menurut-reku/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/apa-itu-stablecoin-crypto-wusd-dalam-konteks-masa-depan-pembayaran-digital/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/grayscale-transfer-us1-milyar-eth-siap-tawarkan-etf-ethereum/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-token-meme-wif-melonjak-akankah-mencapai-us350/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dogecoin-siap-melejit-harga-diprediksi-naik-ke-us022/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/mungkinkah-as-menjadikan-bitcoin-sebagai-aset-cadangan-resmi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-bitcoin-jatuh-memicu-kepanikan-dan-likuidasi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-xrp-berpotensi-untuk-melesat-ini-prediksi-terbarunya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/vaneck-prediksi-bitcoin-capai-us3-juta-di-tahun-2050/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/shiba-eternity-bertransformasi-game-web3-dari-shiba-inu/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/cardano-siap-hadapi-era-voltaire-dengan-hard-fork-chang/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dompet-baru-dan-alamat-aktif-xrp-ledger-capai-rekor-baru-xrp-siap-melesat/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ekosistem-egoncoin-egon-sebagai-public-blockchain-hingga-rumah-bagi-nft-marketplace/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/tren-bullish-notcoin-dan-jasmycoin-sementara-blockdag-jalin-kemitraan-strategis/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/whale-jual-170-milyar-pepe-coin-bagaimana-nasib-token-ini/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/bitcoin-kembali-diserok-para-pemegang-jangka-panjang-tanda-bullish/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-xrp-siap-terbang-tinggi-analis-prediksi-lonjakan-besar/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/apakah-toncoin-sudah-overvalued-ini-kata-para-ahli/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/shiba-inu-di-ambang-pergerakan-besar-bersiap-meroket/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/pidato-pro-kripto-donald-trump-pecat-ketua-sec-dan-bitcoin-jadi-cadangan-as/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/etf-ethereum-grayscale-alami-arus-keluar-besar-besaran-ada-apa/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/transaksi-ada-melejit-whales-bersiap-untuk-hard-fork-cardano/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/kripto-homs-token-untuk-perumahan-usung-transparansi-transaksi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/gebrakan-aeternity-menjawab-tantangan-skalabilitas-blockchain-layer-1/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/meme-coin-fomo-fox-apakah-sekadar-hype-semata/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/minat-institusi-meningkat-apakah-harga-eth-akan-naik/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ripple-siap-buka-1-milyar-xrp-lagi-pada-1-agustus-apa-dampaknya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/whale-dogecoin-serok-serok-lagi-akankah-doge-tembus-us2/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/nansen-saatnya-investor-bitcoin-adopsi-risk-on/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/peran-token-gaugefield-gaui-dalam-ekosistem-stablecoin-gaugecash-gau/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/presale-token-meme-shiba-shootout-tembus-us800-ribu-usung-game-play-to-earn-shiba-sharpshooter/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/situs-blockchain-explorer-blockchair-dukung-42-blockchain-dan-usung-ai-untuk-on-chain-data/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/pemerintah-as-gerakkan-us2-milyar-bitcoin-pasar-panik/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/sentimen-bullish-analis-optimis-harga-xrp-akan-lampaui-ekspektasi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ada-meme-coin-pepu-diklaim-sebagai-pepe-versi-blockchain-layer-2/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/polygon-terus-terjebak-dalam-konsolidasi-inilah-sebabnya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/aktivitas-whale-ethereum-naik-kinerja-etf-ether-jadi-sorotan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/cco-reku-performa-etf-bitcoin-dorong-pertumbuhan-industri-kripto-indonesia/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/upbit-derivatif-kripto-aspek-penting-perkembangan-industri-kripto-indonesia-berikutnya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/presale-token-situs-ternama-99bitcoins-99btc-segera-berakhir/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/blockdag-perkenalkan-tim-eksekutif-baru-dan-rilis-dev-82/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/adu-kuat-pertumbuhan-koin-meme-bonk-cardano-hingga-moonhop/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/antony-turner-sosok-ceo-baru-proyek-kripto-blockdag/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/kiat-moonhop-saingi-kinerja-koin-meme-pepe-dan-wif/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analis-kripto-reku-pertengahan-tahun-kinerja-bitcoin-cenderung-menurun/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/playdoge-play-usung-game-seperti-tamagotchi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-xrp-melesat-37-persen-langkah-awal-menuju-bulan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dominasi-bitcoin-meningkat-altcoin-siap-untuk-melejit/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/blackrock-etf-bitcoin-dan-ethereum-siap-hiasi-portofolio-model-2025/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/toncoin-melonjak-usai-kerja-sama-dengan-trust-wallet-lanjut-bullish/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ini-alasan-pepu-usung-teknologi-blockchain-baru-ini/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/unlimit-berpadu-di-world-app-dan-mendukung-blockchain-world-chain-wld/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/openeden-tokenisasi-t-bills-as-di-xrp-ledger-langkah-besar/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ada-peningkatan-arus-keluar-bursa-bitcoin-jadi-tanda-bullish/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/buidl-ethereum-besutan-blackrock-capai-us522-juta-dividen-meningkat/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-shiba-inu-bisa-naik-drastis-jika-bitcoin-tembus-us3-juta/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/daftar-aset-kripto-berpotensi-positif-jika-the-fed-pangkas-suku-bunga-pada-september/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/spekulasi-penyebab-harga-eth-redup-lagi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/keberhasilan-blockdag-hiasi-pasar-kripto-pembanding-untuk-kondisi-avax-dan-hbar/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/komparasi-meme-coin-pepe-wif-dan-moonhop/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/prediksi-terbaru-analis-dogecoin-bisa-naik-7-200-persen/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ada-akumulasi-whale-besar-besaran-ethereum-siap-naik/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/pasar-berdarah-analis-prediksi-ada-kenaikan-besar-untuk-cardano/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/bitcoin-hadapi-ujian-berat-berisiko-turun-ke-us42-000/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-solana-sol-anjlok-akibat-skandal-ftx-apa-yang-terjadi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/donald-trump-usulkan-bitcoin-untuk-atasi-utang-nasional-as-menarik/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/fbi-keluarkan-peringatkan-untuk-pengguna-kripto-waspadai-penipuan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ripple-melepas-500-juta-xrp-pasar-bergemuruh/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/bitcoin-anjlok-ke-bawah-us55-000-peluang-beli-atau-bahaya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/pasar-ambruk-dogecoin-diprediksi-siap-lari-kencang-dan-melesat-400-persen/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dominasi-bitcoin-bakal-naik-altcoin-diprediksi-dalam-bahaya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/alamat-dompet-baru-ethereum-alami-penurunan-ada-apa/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/hedge-fund-besar-di-eropa-taruh-us500-juta-di-etf-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/donald-trump-tiongkok-akan-kalahkan-as-jika-kita-tidak-bertindak/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/permintaan-xrp-ripple-meningkat-setelah-keputusan-pengadilan-siap-melejit/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/shiba-inu-melesat-dan-ungguli-avalanche-pasar-kripto-bergejolak/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ramalan-terbaru-potensi-the-fed-pangkas-suku-bunga-lagi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/alasan-dunia-tak-perlu-uang-digital-bank-sentral-cbdc/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ikut-rusia-el-salvador-akan-adopsi-kripto-untuk-perdagangan-internasional/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/investor-jangka-panjang-tetap-tenang-saat-bitcoin-anjlok-pertanda-apa/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/whale-xrp-kembali-beraksi-akumulasi-koin-di-harga-rendah/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/solana-tunjukkan-ketangguhan-di-tengah-crash-pasar-terbaru/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/notcoin-melonjak-2493-persen-akankah-tren-ini-berlanjut/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/donald-trump-dijadwalkan-tampil-bersama-elon-musk-topik-kripto-dimunculkan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/giliran-putra-donald-trump-mengaku-menyukai-kripto-demi-politik-semata/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ajaib-kripto-bitcoin-berpotensi-merekah-lagi-ke-us60-ribu/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/pasar-kripto-sempat-extreme-fear-reku-itu-sinyal-pembelian/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/matic-dan-link-hadapi-tantangan-pasar-blockdag-masih-unjuk-gigi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-xrp-meroket-26-persen-usai-ripple-menang-melawan-gugatan-sec/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/binance-raih-rekor-aliran-masuk-us12-milyar-di-tengah-volatilitas/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/hayes-bitcoin-bisa-naik-saat-pair-dolar-yen-berubah/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dogwifhat-wif-naik-13-persen-dalam-sehari-apa-yang-terjadi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/putin-legalkan-penambangan-kripto-langkah-besar-untuk-rusia/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/bitcoin-naik-ke-us58-000-berkat-data-pengangguran-as-yang-membaik/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/binance-akhirnya-listing-toncoin-harga-ton-terbang-melesat/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/shiba-inu-shib-anjlok-21-persen-aktivitas-whale-turun-drastis/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/setelah-kemenangan-hukum-xrp-bersiap-untuk-etf/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-shiba-inu-naik-setelah-burn-rate-meroket-137-000-persen/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ojk-resmi-luncurkan-peta-jalan-iakd-2024-2028-untuk-aset-keuangan-digital-dan-aset-kripto/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/opsi-etf-bitcoin-segera-hadir-q4-2024-siapkan-strategi-investasi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/jaringan-ton-gandeng-intotheblock-wawasan-komprehensif-gratis/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/profesor-as-bitcoin-lebih-baik-daripada-emas/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analis-shiba-inu-siap-melesat-696-persen-di-tahun-2024/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/volume-perdagangan-kripto-di-bursa-melonjak-19-persen-apa-penyebabnya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/bitcoin-membentuk-local-low-ketiga-di-2024-rebound-di-depan-mata/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-xrp-di-ambang-breakout-besar-apa-yang-bisa-terjadi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/pasar-aset-kripto-juli-2024-riset-coinex-volatilitas-tinggi-lalu-pulih-signifikan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/marathon-digital-kumpulkan-us250-juta-demi-serok-bitcoin-lagi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/pasar-opsi-xrp-meledak-akankah-etf-segera-datang/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-toncoin-naik-lagi-akankah-melesat-dan-tembus-us9/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dogecoin-naik-signifikan-akumulasi-whale-doge-meningkat/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/usai-kripto-kaspa-kas-sentuh-ath-dan-blockdag-network-bdag-raih-us653-juta/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/begini-prediksi-harga-ethereum-eth-setelah-rilis-data-cpi-as/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/arthur-hayes-bull-run-kripto-akan-dimulai-pada-september-2024/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/shiba-inu-kembali-melejit-lonjakan-240-persen-dalam-transaksi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dominasi-bitcoin-terus-naik-di-tahun-2024-ada-apa/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/membaca-peluang-kenaikan-lanjutan-bitcoin-dan-altcoins-di-minggu-ini/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ini-imbas-data-inflasi-ke-pasar-kripto-bitcoin-akan-baik-baik-saja/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/konferensi-kripto-webx2024-akan-hadirkan-perdana-menteri-jepang/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dana-segar-stablecoin-us28-miliar-siap-masuk-pasar-kripto-reli-bitcoin-bisa-berlanjut/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/metamask-gandeng-mastercard-perkenalkan-kartu-debit-blockchain/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/bitnomial-rilis-xrp-berjangka-etf-xrp-di-depan-mata/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/tingkat-pendanaan-ethereum-turun-sentimen-bearish-dominan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/bitcoin-dilanda-krisis-likuiditas-dampak-dari-etf-spot-as/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/kamala-harris-isyaratkan-ingin-pro-aset-kripto-lewat-penunjukan-penasihat-baru/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/koreksi-harga-pepe-akan-berlanjut/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/tren-investasi-etf-bitcoin-institusional-makin-percaya-diri/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/xrp-menghadapi-likuiditas-yang-lambat-volume-perdagangan-anjlok/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/shibarium-siap-luncurkan-liquid-staking-inilah-tanggalnya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-dogwifhat-wif-merosot-tajam-peluang-beli-atau-perlu-waspada/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/prediksi-gila-harga-xrp-tembus-us57-263-dan-perkembangan-terkini-blockdag-network/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/momentum-kuat-harga-trx-dan-blockdag-network-sasar-liga-champions/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/cardano-melejit-lonjakan-68-persen-dalam-volume-perdagangan-ada/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/integrasi-injective-dan-ton-blockchain-bawa-peluang-baru-di-defi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/di-dalam-pola-ascending-triangle-apakah-harga-bnb-siap-melejit/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/blackrock-geser-grayscale-sebagai-raja-etf-kripto/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analis-prediksi-reli-kuat-pada-harga-shib-di-2025-berapa-targetnya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/paypal-dorong-pertumbuhan-pyusd-di-solana-dengan-insentif-baru/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dogecoin-melonjak-pengguna-aktif-naik-32-persen-dalam-sehari/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/etf-bitcoin-minat-institusi-tumbuh-27-persen-di-kuartal-kedua/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analisis-ai-beberkan-10-fakta-solana-apakah-masih-layak-diinvestasikan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/lace-paper-wallet-bounty-us1-juta-dari-pendiri-cardanolace-paper-wallet-bounty-1-juta-dari-pendiri-cardano/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/shiba-inu-hadapi-penurunan-peluang-akumulasi-muncul/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/sinyal-bullish-muncul-bitcoin-siap-kembali-ke-us60-000/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/lufina-hadirkan-game-berhadiah-kripto-fina-setara-us1-juta/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/tekanan-jual-pepe-semakin-kuat-dan-potensi-naik-105-persen-berdasarkan-analisis-ini/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ekonomi-as-tangguh-analis-reku-imbas-ke-pasar-kripto-belum-cukup-positif/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
]

all_articles = []

# Collect and summarize data from each website
for website in websites:
    article = scrape_website(
        website['url'], 
        website['title_tag'], 
        website['title_class'], 
        website['date_tag'], 
        website['date_class'], 
        website['content_tag'], 
        website['content_class']
    )
    all_articles.append(article)

# Save summarized data to CSV file
csv_file = './preprocessed/blockchainmedia.csv'
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['Title', 'Date', 'Summary'])
    writer.writeheader()
    for article in all_articles:
        writer.writerow(article)

# Print the contents of the CSV file
print(f"Data has been saved to {csv_file}")

df = pd.read_csv(csv_file)
print(df)

Data has been saved to ./preprocessed/blockchainmedia.csv
                                                 Title            Date  \
0    Michael Saylor Tetap Optimis Meski Harga BTC M...    23 June 2024   
1    Masa Depan Toncoin: Harga TON di Ambang Kebang...    23 June 2024   
2    Think Tank Korea Selatan Ragukan ETF Bitcoin, ...    24 June 2024   
3    Dogwifhat (WIF) Keluar dari 50 Besar, Harga An...    24 June 2024   
4    XRP Ripple Terus Berkonsolidasi, Kapan Harga A...    24 June 2024   
..                                                 ...             ...   
291  Shiba Inu Hadapi Penurunan, Peluang Akumulasi ...  19 August 2024   
292  Sinyal Bullish Muncul! Bitcoin Siap Kembali ke...  19 August 2024   
293  Lufina Hadirkan Game Berhadiah Kripto FINA Set...  19 August 2024   
294  Tekanan Jual PEPE Semakin Kuat dan Potensi Nai...  19 August 2024   
295  Ekonomi AS Tangguh, Analis Reku: Imbas ke Pasa...  19 August 2024   

                                               Summar

In [1]:
from bs4 import BeautifulSoup
import requests
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
import torch.nn as nn
import re
import csv
import pandas as pd

# Initialize the BERTSUM model and tokenizer
class BERTSUM(nn.Module):
    def __init__(self, model_name='bert-base-uncased'):
        super(BERTSUM, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.linear = nn.Linear(self.bert.config.hidden_size, 1)
        self.softmax = nn.Softmax(dim=1)  # Per word softmax for importance

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        word_output = outputs.last_hidden_state  # Word token representations
        scores = self.linear(word_output)
        scores = self.softmax(scores)
        return scores.squeeze(-1)

# Load pre-trained model and tokenizer
model = BERTSUM()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Fungsi untuk ekstraksi embedding kalimat menggunakan BERT
def get_sentence_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors="pt")
    outputs = model.bert(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.detach().numpy().flatten()

# Fungsi untuk melakukan rangkuman ekstraktif berbasis cosine similarity
def extractive_summary(text, ratio=0.7):
    sentences = text.split('. ')
    sentence_embeddings = [get_sentence_embedding(sentence) for sentence in sentences]

    # Menghitung similaritas kosinus antar kalimat
    similarities = cosine_similarity(sentence_embeddings)
    sentence_scores = similarities.sum(axis=1)

    # Menentukan jumlah kalimat berdasarkan ratio
    top_n = max(1, int(len(sentences) * ratio))
    top_sentence_indices = np.argsort(sentence_scores)[-top_n:]
    top_sentence_indices.sort()

    summary = '. '.join([sentences[i] for i in top_sentence_indices])
    return summary

# Function to extract important sentences based on word importance scores using BERTSUM
def summarize_text(text, model, tokenizer, num_sentences=1):
    paragraphs = text.split('\n')
    paragraphs = [p for p in paragraphs if p.strip()]
    important_sentences = []
    
    for paragraph in paragraphs:
        sentences = re.split(r'(?<=[.!?]) +', paragraph)
        paragraph_summary = []

        for sentence in sentences:
            inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
            
            if 'token_type_ids' in inputs:
                del inputs['token_type_ids']
            
            tokenized_words = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
            
            with torch.no_grad():
                scores = model(**inputs)
            
            if scores.numel() > 0:
                k = min(5, scores.size(0))
                top_word_indices = torch.topk(scores.squeeze(), k=k).indices
                top_words = [tokenized_words[i.item()] for i in top_word_indices if tokenized_words[i.item()] not in tokenizer.all_special_tokens]

                if len(top_words) > 0:
                    paragraph_summary.append(sentence)

            else:
                print("Scores tensor is empty. Skipping this sentence.")

        if paragraph_summary:
            important_sentences.append(paragraph_summary[0])

    summary = ' '.join(important_sentences)
    return summary

# Function to scrape website and summarize
def scrape_website(url, title_tag, title_class, date_tag, date_class, content_tag, content_class, summary_type='bert', ratio=0.7):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    title_element = soup.find(title_tag, class_=title_class)
    date_element = soup.find(date_tag, class_=date_class)
    content_element = soup.find(content_tag, class_=content_class)

    title = title_element.get_text(strip=True) if title_element else "No Title Found"
    date = date_element.get_text(strip=True) if date_element else "No Date Found"
    content = content_element.get_text(strip=True) if content_element else "No Content Found"

    # Memilih metode rangkuman berdasarkan summary_type
    if summary_type == 'bert':
        summary = summarize_text(content, model, tokenizer, num_sentences=1)
    elif summary_type == 'cosine':
        summary = extractive_summary(content, ratio=0.7)
    else:
        raise ValueError("Invalid summary_type. Choose either 'bert' or 'cosine'.")

    return {'Title': title, 'Date': date, 'Summary': summary}

# URL and HTML tag details for the website
websites = [
    {
        'url': 'https://blockchainmedia.id/michael-saylor-tetap-optimis-meski-harga-btc-macet/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/masa-depan-toncoin-harga-ton-di-ambang-kebangkitan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/think-tank-korea-selatan-ragukan-etf-bitcoin-ada-apa/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dogwifhat-wif-keluar-dari-50-besar-harga-anjlok-38-persen/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/xrp-ripple-terus-berkonsolidasi-kapan-harga-akan-naik/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/minat-pada-shiba-inu-meningkat-biaya-shibarium-meroket/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/prospek-titik-balik-bch-prediksi-bullish-toncoin-ton-hingga-inovasi-platform-blockdag/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/bagaimana-pertumbuhan-ethereum-eth-dan-tren-pasar-bitcoin-btc-picu-peningkatan-presale-furrever-token/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/pasar-etf-bitcoin-spot-masih-merah-sol-meluncur-di-bawah-ma200-dan-rollblock-rblk-coba-menarik-perhatian/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/uniswap-dan-etc-alami-beda-nasib-influencer-ulas-teknologi-blockdag/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/mt-gox-akan-mulai-bayar-bitcoin-juli-2024-kreditur-bersorak/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-doge-dalam-zona-beli-kesempatan-emas-bagi-investor/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/volume-toncoin-naik-drastis-apa-artinya-bagi-pasar/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/scaramucci-biden-menang-harga-bitcoin-bisa-capai-us250-000/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analisis-harga-pepe-shib-dan-prospek-baru-rollblock-rblk/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ketokohan-gary-gensler-di-balik-penerbitan-meme-coin-sec-berbasis-blockchain-ripple-xrp/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/5-crypto-mining-pilihan-blockdag-hingga-zcash/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analis-popular-bitcoin-akan-melejit-ke-us70-000-dalam-waktu-dekat/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/sec-gensler-etf-ethereum-berjalan-lancar-dan-mendapat-dukungan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/solana-hadapi-rival-baru-potensi-reli-sui-hingga-teknologi-canggih-blockdag/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-wif-lesu-pasca-akumulasi-whale-bagaimana-prediksinya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/willy-woo-bitcoin-akan-menyaingi-dolar-as-di-masa-depan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/whale-dogecoin-serok-serok-di-tengah-pasar-yang-lesu-tanda-apakah-ini/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/altseason-altcoin-siap-menggebrak-meski-bitcoin-anjlok/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/bear-di-bitcoin-masih-pegang-kendali-perdagangan-etf-ethereum-spot-diprediksi-juli-dan-rollblock-rblk-masuki-kripto-berbasis-permainan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/monetisasi-data-perjalanan-dan-token-ptrn-dari-pathearn/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ekspansi-ripple-stablecoin-baru-dan-peluang-etf-xrp/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ringkasan-berita-kripto-hari-ini-setelah-burn-not-us3-juta-vaneck-ajukan-proposal-etf-sol-potensi-bullish-xrp-serta-ulasan-rblk-di-sektor-permainan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ringkasan-berita-kripto-hari-ini-setelah-burn-not-us3-juta-vaneck-ajukan-proposal-etf-sol-potensi-bullish-xrp-serta-ulasan-rblk-di-sektor-permainan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/vaneck-ajukan-etf-solana-spot-pertama-dunia-kripto-berguncang-analis-prediksi-meluncur-tahun-2025/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/token-notcoin-not-melejit-usai-burn-apa-selanjutnya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/siapa-susul-proposal-etf-solana-spot-prediksi-gerakan-harga-btc-berikutnya-dan-jejak-awal-rollblock-rblk-di-industri-hiburan-permainan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analisis-pertumbuhan-toncoin-ton-render-rndr-hingga-blockdag/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ulasan-terbaru-inovasi-dan-pertumbuhan-blockdag/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/1-juta-alamat-kini-memegang-1-bitcoin-btc-meski-pasar-bergejolak/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-notcoin-uji-level-kritis-pemulihan-di-depan-mata/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/etf-ethereum-bisa-dorong-harga-eth-melesat-ke-us6-500-ini-alasannya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/kembali-berulah-whale-pindahkan-155-triliun-shib-untuk-apa/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/berita-kripto-terkini-rumor-penundaan-persetujuan-perdagangan-etf-ethereum-usai-harga-tia-luruh-46-persen-dan-rollblock-cicipi-industri-permainan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/prediksi-harga-btc-terbaru-bisa-terjungkal-ke-us55-ribu-dan-gambaran-testnet-blockchain-blockdag/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analis-harga-xrp-bisa-melesat-ke-us070-ini-alasannya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/volume-cardano-ada-meroket-di-tengah-penurunan-pasar-kripto-ada-apa/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analisis-etf-solana-dan-potensi-lonjakan-harga-yang-menggiurkan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/kapitulasi-penambang-bitcoin-meningkat-sejak-halving-april-ada-apa/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analisis-teknikal-kripto-kaspa-kas-usai-cetak-ath-dan-rollblock-yang-berfitur-buy-back/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/bitcoin-anjlok-dua-kali-begini-analisis-pasarnya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-ethereum-eth-melonjak-siap-untuk-dorongan-bullish-lainnya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/investor-kembali-akumulasi-shib-harga-siap-pulih/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dogecoin-beri-tanda-apakah-harga-akan-segera-naik-lagi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/sony-akuisisi-crypto-exchange-pepe-menguat-lagi-hari-ini-dan-peran-rollblock-rblk-di-industri-hiburan-dan-permainan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/solana-bersiap-jadi-kripto-terbesar-ketiga-tahun-ini-ini-alasannya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/solana-bersiap-jadi-kripto-terbesar-ketiga-tahun-ini-ini-alasannya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/solana-bersiap-jadi-kripto-terbesar-ketiga-tahun-ini-ini-alasannya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/solana-bersiap-jadi-kripto-terbesar-ketiga-tahun-ini-ini-alasannya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/perbendaharaan-polkadot-dalam-kondisi-krisis-ini-faktanya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ripple-pindahkan-1-milyar-xrp-dari-dompet-escrow-untuk-apa/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ripple-pindahkan-1-milyar-xrp-dari-dompet-escrow-untuk-apa/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ripple-pindahkan-1-milyar-xrp-dari-dompet-escrow-untuk-apa/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analis-harga-doge-akan-mendaki-ke-us1-dalam-siklus-bull-terbaru/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/aktivitas-pengembangan-cardano-merosot-ada-apa/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/mengenal-dasia-platform-pemasaran-digital-berbasis-teknologi-ai-dan-blockchain/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ulasan-kripto-hari-ini-prediksi-xrp-us144-nasib-eth-berikutnya-dan-rollblock-bertenaga-ai/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ulasan-kripto-hari-ini-prediksi-xrp-us144-nasib-eth-berikutnya-dan-rollblock-bertenaga-ai/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/penyebab-bitcoin-turun-menghujam-us60-ribu-lagi-dan-gambaran-rollblock-rblk-yang-diperkuat-kecerdasan-buatan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analis-reku-ada-indikasi-strong-buy-untuk-altcoin/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/bitwise-etf-ethereum-bisa-debut-pertengahan-juli/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/sentimen-bearish-meningkat-harga-floki-terancam-turun-lagi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-xrp-digadang-siap-meroket-hingga-7-600-persen/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/robert-kiyosaki-prediksi-crash-besar-dan-kebangkitan-harga-btc-ke-us10-juta/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-btc-terjun-bebas-ke-us53-571-jauh-di-bawah-ema-200-dan-rollblock-tawarkan-permainan-beragam/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dogecoin-alami-penurunan-besar-inilah-penyebabnya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-xrp-terus-turun-kejelasan-hukum-jadi-pertanyaan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-xrp-terus-turun-kejelasan-hukum-jadi-pertanyaan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/whale-antusias-menyambut-fetch-ai-fet-berubah-jadi-asi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ada-antusiasme-baru-terhadap-etf-ethereum-di-pemilu-as-pertanda-baik/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-shib-rebound-15-persen-apakah-ini-awal-kenaikan-baru/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-eth-berpeluang-naik-dan-saingi-btc-etf-ethereum-segera-hadir/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-eth-berpeluang-naik-dan-saingi-btc-etf-ethereum-segera-hadir/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-eth-berpeluang-naik-dan-saingi-btc-etf-ethereum-segera-hadir/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/worldcoin-mencoba-pulih-pembalikan-harga-segera-terjadi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-shiba-inu-melonjak-burn-shib-melejit-781-persen/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/prediksi-baru-bitcoin-bisa-meroket-ke-us225-000-dalam-beberapa-bulan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-xrp-to-the-moon-spekulasi-meningkat-setelah-postingan-bitstamp/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/etf-solana-di-ambang-persetujuan-apa-dampaknya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/prediksi-harga-shiba-inu-shib-bisa-melesat-hingga-1-487-persen/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analis-harga-bitcoin-perlu-mencapai-us50-000-sebelum-kembali-dalam-lompatan-bullish/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/prediksi-harga-doge-bisakah-mempertahankan-momentumnya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/worldcoin-perkenalkan-world-chain-jaringan-layer-2-terbaru/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/etf-bitcoin-as-cetak-rekor-arus-masuk-di-tengah-pasar-lesu-blackrock-memimpin/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/pengguna-ripple-meningkat-volume-transaksi-sentuh-us1-milyar/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/burn-rate-shiba-inu-shib-melejit-347-persen-dalam-sehari/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/bitcoin-kesulitan-di-us60-000-apa-yang-terjadi-selanjutnya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/solana-siap-melejit-harga-sol-diprediksi-sentuh-us600/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dominasi-bitcoin-tinggi-altseason-masih-bisa-diharapkan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/aktivitas-shibarium-turun-bagaimana-nasib-harga-shib/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/etf-ethereum-diprediksi-menarik-us10-milyar-arus-masuk-eth-siap-meroket/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/korelasi-bitcoin-dan-sp-500-kian-renggang-apa-penyebabnya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/masa-depan-ripple-dapatkah-pemilu-as-mengubah-permainan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/gaming-token-superblok-sebagai-cara-baru-untuk-game-dan-peluang-menambah-pendapatan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/perkembangan-terkini-kripto-ripple-xrp-hijau-benderang-nasib-pepecoin-pepe-serta-presale-furrever-token-mendekati-tahap-akhir/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/varia-kabar-kripto-peluang-crossover-bullish-eth-potensi-doge-luruh-lagi-dan-furrever-token-segera-di-pancakeswap/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ajaib-kripto-bitcoin-masih-berpotensi-kembali-melemah-ke-us54-ribu/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/aset-kripto-di-tengah-anomali-pasca-rilis-data-cpi-dan-potensi-the-fed-pangkas-suku-bunga-di-september/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/etf-bitcoin-bangkit-arus-masuk-dana-meningkat-hebat/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/whale-litecoin-menggila-transaksi-us285-milyar-setiap-hari/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/cardano-naik-22-persen-dalam-seminggu-harga-ada-siap-melejit-lebih-tinggi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/worldcoin-siap-mempercepat-pembukaan-tokennya-apa-dampaknya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-xrp-naik-pesat-kepercayaan-investor-menguat-lagi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-xrp-naik-pesat-kepercayaan-investor-menguat-lagi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-toncoin-ton-menguat-lagi-investor-optimis-ath-baru/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-toncoin-ton-menguat-lagi-investor-optimis-ath-baru/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-eth-melonjak-akankah-melesat-lampaui-level-us3-500/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/doge-berjuang-naik-apa-peran-whale-dalam-pergerakan-ini/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/whale-bitcoin-serok-71-000-btc-di-tengah-penurunan-harga/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/sec-diprediksi-akan-setujui-etf-ethereum-minggu-ini/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dompet-whale-yang-tertidur-12-tahun-bangkit-dan-pindahkan-1-000-btc-pertanda-apa/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-xrp-naik-26-persen-spekulasi-penutupan-investigasi-sec-kian-memanas/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ekosistem-shiba-inu-menanjak-aktivitas-whale-jadi-sorotan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ai-meramal-harga-bitcoin-btc-pasca-pembayaran-mt-gox/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/nascent-serok-141-milyar-token-harga-pepe-coin-meroket/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/sambut-etf-ethereum-optimisme-investor-institusi-lebih-tinggi-daripada-ritel/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ada-tanda-bullish-analis-prediksi-harga-sol-tembus-us174/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/konsolidasi-harga-bonk-apakah-penembusan-akan-terjadi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ton-perkenalkan-teleport-btc-jembatan-bitcoin-baru/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/staking-ethereum-capai-rekor-tertinggi-jelang-etf-spot-tanda-bullish/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dogecoin-melampaui-90-juta-dompet-siap-melejit-ke-us1/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/strategi-baru-ripple-menghindari-ipo-fokus-tender-saham/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dogwifhat-wif-mendaki-kuat-kembali-ke-50-kripto-teratas/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/bitcoin-diproyeksi-melonjak-ke-us200-000-setelah-kapitulasi-penambang/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/polygon-siap-bertransformasi-migrasi-koin-matic-ke-pol/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ada-pola-bullish-pennant-sinyal-melesat-untuk-harga-sol/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/sistem-windows-crash-bitcoin-dan-blockchain-tetap-berfungsi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/whale-ethereum-serok-serok-harga-eth-siap-melonjak/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dogecoin-turun-lagi-whale-mulai-borong-doge-lagi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/prediksi-bullish-cardano-harga-ada-siap-menuju-us1/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/arus-masuk-etf-bitcoin-naik-institusi-kian-berminat/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-xrp-melonjak-harapan-baru-dari-pertemuan-sec/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/prediksi-para-ahli-shiba-inu-berpotensi-naik-ke-us0000045/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analis-senior-peter-brandt-dan-prediksi-bearish-pada-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/melesat-harga-polkadot-dot-berpotensi-naik-hingga-227-persen/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ada-transaksi-50-triliun-shiba-inu-shib-imbas-peretasan-wazirx/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/garlinghouse-pertempuran-ripple-vs-sec-akan-segera-berakhir/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analisis-harga-xrp-siap-untuk-kenaikan-besar-melawan-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/etf-ethereum-resmi-disetujui-sec-siap-diperdagangkan-hari-ini/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dogecoin-mendaki-tinggi-ada-potensi-kenaikan-harga-lagi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ibit-blackrock-menerima-dana-masuk-us523-juta-bitcoin-melonjak/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/tren-kripto-terkini-momentum-filecoin-dan-tron-bagaimana-dengan-blockdag/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/prediksi-harga-blockdag-hingga-dinamika-pasar-celestia-dan-algorand/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/cfx-perkuat-komitmen-pengembangan-pasar-kripto-indonesia-ini-respons-bappebti/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/perdagangan-etf-spot-ethereum-dimulai-hari-ini-berikut-potensinya-menurut-reku/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/apa-itu-stablecoin-crypto-wusd-dalam-konteks-masa-depan-pembayaran-digital/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/grayscale-transfer-us1-milyar-eth-siap-tawarkan-etf-ethereum/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-token-meme-wif-melonjak-akankah-mencapai-us350/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dogecoin-siap-melejit-harga-diprediksi-naik-ke-us022/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/mungkinkah-as-menjadikan-bitcoin-sebagai-aset-cadangan-resmi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-bitcoin-jatuh-memicu-kepanikan-dan-likuidasi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-xrp-berpotensi-untuk-melesat-ini-prediksi-terbarunya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/vaneck-prediksi-bitcoin-capai-us3-juta-di-tahun-2050/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/shiba-eternity-bertransformasi-game-web3-dari-shiba-inu/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/cardano-siap-hadapi-era-voltaire-dengan-hard-fork-chang/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dompet-baru-dan-alamat-aktif-xrp-ledger-capai-rekor-baru-xrp-siap-melesat/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ekosistem-egoncoin-egon-sebagai-public-blockchain-hingga-rumah-bagi-nft-marketplace/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/tren-bullish-notcoin-dan-jasmycoin-sementara-blockdag-jalin-kemitraan-strategis/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/whale-jual-170-milyar-pepe-coin-bagaimana-nasib-token-ini/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/bitcoin-kembali-diserok-para-pemegang-jangka-panjang-tanda-bullish/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-xrp-siap-terbang-tinggi-analis-prediksi-lonjakan-besar/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/apakah-toncoin-sudah-overvalued-ini-kata-para-ahli/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/shiba-inu-di-ambang-pergerakan-besar-bersiap-meroket/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/pidato-pro-kripto-donald-trump-pecat-ketua-sec-dan-bitcoin-jadi-cadangan-as/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/etf-ethereum-grayscale-alami-arus-keluar-besar-besaran-ada-apa/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/transaksi-ada-melejit-whales-bersiap-untuk-hard-fork-cardano/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/kripto-homs-token-untuk-perumahan-usung-transparansi-transaksi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/gebrakan-aeternity-menjawab-tantangan-skalabilitas-blockchain-layer-1/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/meme-coin-fomo-fox-apakah-sekadar-hype-semata/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/minat-institusi-meningkat-apakah-harga-eth-akan-naik/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ripple-siap-buka-1-milyar-xrp-lagi-pada-1-agustus-apa-dampaknya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/whale-dogecoin-serok-serok-lagi-akankah-doge-tembus-us2/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/nansen-saatnya-investor-bitcoin-adopsi-risk-on/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/peran-token-gaugefield-gaui-dalam-ekosistem-stablecoin-gaugecash-gau/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/presale-token-meme-shiba-shootout-tembus-us800-ribu-usung-game-play-to-earn-shiba-sharpshooter/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/situs-blockchain-explorer-blockchair-dukung-42-blockchain-dan-usung-ai-untuk-on-chain-data/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/pemerintah-as-gerakkan-us2-milyar-bitcoin-pasar-panik/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/sentimen-bullish-analis-optimis-harga-xrp-akan-lampaui-ekspektasi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ada-meme-coin-pepu-diklaim-sebagai-pepe-versi-blockchain-layer-2/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/polygon-terus-terjebak-dalam-konsolidasi-inilah-sebabnya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/aktivitas-whale-ethereum-naik-kinerja-etf-ether-jadi-sorotan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/cco-reku-performa-etf-bitcoin-dorong-pertumbuhan-industri-kripto-indonesia/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/upbit-derivatif-kripto-aspek-penting-perkembangan-industri-kripto-indonesia-berikutnya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/presale-token-situs-ternama-99bitcoins-99btc-segera-berakhir/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/blockdag-perkenalkan-tim-eksekutif-baru-dan-rilis-dev-82/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/adu-kuat-pertumbuhan-koin-meme-bonk-cardano-hingga-moonhop/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/antony-turner-sosok-ceo-baru-proyek-kripto-blockdag/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/kiat-moonhop-saingi-kinerja-koin-meme-pepe-dan-wif/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analis-kripto-reku-pertengahan-tahun-kinerja-bitcoin-cenderung-menurun/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/playdoge-play-usung-game-seperti-tamagotchi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-xrp-melesat-37-persen-langkah-awal-menuju-bulan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dominasi-bitcoin-meningkat-altcoin-siap-untuk-melejit/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/blackrock-etf-bitcoin-dan-ethereum-siap-hiasi-portofolio-model-2025/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/toncoin-melonjak-usai-kerja-sama-dengan-trust-wallet-lanjut-bullish/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ini-alasan-pepu-usung-teknologi-blockchain-baru-ini/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/unlimit-berpadu-di-world-app-dan-mendukung-blockchain-world-chain-wld/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/openeden-tokenisasi-t-bills-as-di-xrp-ledger-langkah-besar/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ada-peningkatan-arus-keluar-bursa-bitcoin-jadi-tanda-bullish/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/buidl-ethereum-besutan-blackrock-capai-us522-juta-dividen-meningkat/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-shiba-inu-bisa-naik-drastis-jika-bitcoin-tembus-us3-juta/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/daftar-aset-kripto-berpotensi-positif-jika-the-fed-pangkas-suku-bunga-pada-september/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/spekulasi-penyebab-harga-eth-redup-lagi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/keberhasilan-blockdag-hiasi-pasar-kripto-pembanding-untuk-kondisi-avax-dan-hbar/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/komparasi-meme-coin-pepe-wif-dan-moonhop/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/prediksi-terbaru-analis-dogecoin-bisa-naik-7-200-persen/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ada-akumulasi-whale-besar-besaran-ethereum-siap-naik/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/pasar-berdarah-analis-prediksi-ada-kenaikan-besar-untuk-cardano/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/bitcoin-hadapi-ujian-berat-berisiko-turun-ke-us42-000/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-solana-sol-anjlok-akibat-skandal-ftx-apa-yang-terjadi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/donald-trump-usulkan-bitcoin-untuk-atasi-utang-nasional-as-menarik/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/fbi-keluarkan-peringatkan-untuk-pengguna-kripto-waspadai-penipuan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ripple-melepas-500-juta-xrp-pasar-bergemuruh/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/bitcoin-anjlok-ke-bawah-us55-000-peluang-beli-atau-bahaya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/pasar-ambruk-dogecoin-diprediksi-siap-lari-kencang-dan-melesat-400-persen/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dominasi-bitcoin-bakal-naik-altcoin-diprediksi-dalam-bahaya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/alamat-dompet-baru-ethereum-alami-penurunan-ada-apa/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/hedge-fund-besar-di-eropa-taruh-us500-juta-di-etf-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/donald-trump-tiongkok-akan-kalahkan-as-jika-kita-tidak-bertindak/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/permintaan-xrp-ripple-meningkat-setelah-keputusan-pengadilan-siap-melejit/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/shiba-inu-melesat-dan-ungguli-avalanche-pasar-kripto-bergejolak/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ramalan-terbaru-potensi-the-fed-pangkas-suku-bunga-lagi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/alasan-dunia-tak-perlu-uang-digital-bank-sentral-cbdc/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ikut-rusia-el-salvador-akan-adopsi-kripto-untuk-perdagangan-internasional/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/investor-jangka-panjang-tetap-tenang-saat-bitcoin-anjlok-pertanda-apa/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/whale-xrp-kembali-beraksi-akumulasi-koin-di-harga-rendah/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/solana-tunjukkan-ketangguhan-di-tengah-crash-pasar-terbaru/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/notcoin-melonjak-2493-persen-akankah-tren-ini-berlanjut/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/donald-trump-dijadwalkan-tampil-bersama-elon-musk-topik-kripto-dimunculkan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/giliran-putra-donald-trump-mengaku-menyukai-kripto-demi-politik-semata/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ajaib-kripto-bitcoin-berpotensi-merekah-lagi-ke-us60-ribu/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/pasar-kripto-sempat-extreme-fear-reku-itu-sinyal-pembelian/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/matic-dan-link-hadapi-tantangan-pasar-blockdag-masih-unjuk-gigi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-xrp-meroket-26-persen-usai-ripple-menang-melawan-gugatan-sec/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/binance-raih-rekor-aliran-masuk-us12-milyar-di-tengah-volatilitas/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/hayes-bitcoin-bisa-naik-saat-pair-dolar-yen-berubah/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dogwifhat-wif-naik-13-persen-dalam-sehari-apa-yang-terjadi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/putin-legalkan-penambangan-kripto-langkah-besar-untuk-rusia/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/bitcoin-naik-ke-us58-000-berkat-data-pengangguran-as-yang-membaik/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/binance-akhirnya-listing-toncoin-harga-ton-terbang-melesat/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/shiba-inu-shib-anjlok-21-persen-aktivitas-whale-turun-drastis/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/setelah-kemenangan-hukum-xrp-bersiap-untuk-etf/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-shiba-inu-naik-setelah-burn-rate-meroket-137-000-persen/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ojk-resmi-luncurkan-peta-jalan-iakd-2024-2028-untuk-aset-keuangan-digital-dan-aset-kripto/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/opsi-etf-bitcoin-segera-hadir-q4-2024-siapkan-strategi-investasi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/jaringan-ton-gandeng-intotheblock-wawasan-komprehensif-gratis/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/profesor-as-bitcoin-lebih-baik-daripada-emas/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analis-shiba-inu-siap-melesat-696-persen-di-tahun-2024/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/volume-perdagangan-kripto-di-bursa-melonjak-19-persen-apa-penyebabnya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/bitcoin-membentuk-local-low-ketiga-di-2024-rebound-di-depan-mata/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-xrp-di-ambang-breakout-besar-apa-yang-bisa-terjadi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/pasar-aset-kripto-juli-2024-riset-coinex-volatilitas-tinggi-lalu-pulih-signifikan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/marathon-digital-kumpulkan-us250-juta-demi-serok-bitcoin-lagi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/pasar-opsi-xrp-meledak-akankah-etf-segera-datang/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-toncoin-naik-lagi-akankah-melesat-dan-tembus-us9/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dogecoin-naik-signifikan-akumulasi-whale-doge-meningkat/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/usai-kripto-kaspa-kas-sentuh-ath-dan-blockdag-network-bdag-raih-us653-juta/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/begini-prediksi-harga-ethereum-eth-setelah-rilis-data-cpi-as/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/arthur-hayes-bull-run-kripto-akan-dimulai-pada-september-2024/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/shiba-inu-kembali-melejit-lonjakan-240-persen-dalam-transaksi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dominasi-bitcoin-terus-naik-di-tahun-2024-ada-apa/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/membaca-peluang-kenaikan-lanjutan-bitcoin-dan-altcoins-di-minggu-ini/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ini-imbas-data-inflasi-ke-pasar-kripto-bitcoin-akan-baik-baik-saja/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/konferensi-kripto-webx2024-akan-hadirkan-perdana-menteri-jepang/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dana-segar-stablecoin-us28-miliar-siap-masuk-pasar-kripto-reli-bitcoin-bisa-berlanjut/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/metamask-gandeng-mastercard-perkenalkan-kartu-debit-blockchain/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/bitnomial-rilis-xrp-berjangka-etf-xrp-di-depan-mata/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/tingkat-pendanaan-ethereum-turun-sentimen-bearish-dominan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/bitcoin-dilanda-krisis-likuiditas-dampak-dari-etf-spot-as/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/kamala-harris-isyaratkan-ingin-pro-aset-kripto-lewat-penunjukan-penasihat-baru/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/koreksi-harga-pepe-akan-berlanjut/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/tren-investasi-etf-bitcoin-institusional-makin-percaya-diri/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/xrp-menghadapi-likuiditas-yang-lambat-volume-perdagangan-anjlok/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/shibarium-siap-luncurkan-liquid-staking-inilah-tanggalnya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/harga-dogwifhat-wif-merosot-tajam-peluang-beli-atau-perlu-waspada/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/prediksi-gila-harga-xrp-tembus-us57-263-dan-perkembangan-terkini-blockdag-network/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/momentum-kuat-harga-trx-dan-blockdag-network-sasar-liga-champions/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/cardano-melejit-lonjakan-68-persen-dalam-volume-perdagangan-ada/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/integrasi-injective-dan-ton-blockchain-bawa-peluang-baru-di-defi/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/di-dalam-pola-ascending-triangle-apakah-harga-bnb-siap-melejit/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/blackrock-geser-grayscale-sebagai-raja-etf-kripto/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analis-prediksi-reli-kuat-pada-harga-shib-di-2025-berapa-targetnya/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/paypal-dorong-pertumbuhan-pyusd-di-solana-dengan-insentif-baru/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/dogecoin-melonjak-pengguna-aktif-naik-32-persen-dalam-sehari/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/etf-bitcoin-minat-institusi-tumbuh-27-persen-di-kuartal-kedua/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/analisis-ai-beberkan-10-fakta-solana-apakah-masih-layak-diinvestasikan/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/lace-paper-wallet-bounty-us1-juta-dari-pendiri-cardanolace-paper-wallet-bounty-1-juta-dari-pendiri-cardano/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/shiba-inu-hadapi-penurunan-peluang-akumulasi-muncul/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/sinyal-bullish-muncul-bitcoin-siap-kembali-ke-us60-000/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/lufina-hadirkan-game-berhadiah-kripto-fina-setara-us1-juta/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/tekanan-jual-pepe-semakin-kuat-dan-potensi-naik-105-persen-berdasarkan-analisis-ini/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
    {
        'url': 'https://blockchainmedia.id/ekonomi-as-tangguh-analis-reku-imbas-ke-pasar-kripto-belum-cukup-positif/',
        'title_tag': 'h1',
        'title_class': 'tdb-title-text',        
        'date_tag': 'time',
        'date_class': 'entry-date updated td-module-date',
        'content_tag': 'div',
        'content_class': 'td_block_wrap tdb_single_content tdi_62 td-pb-border-top td_block_template_1 td-post-content tagdiv-type'
    },
]

all_articles = []

# Collect and summarize data from each website
for website in websites:
    article = scrape_website(
        website['url'], 
        website['title_tag'], 
        website['title_class'], 
        website['date_tag'], 
        website['date_class'], 
        website['content_tag'], 
        website['content_class']
    )
    all_articles.append(article)

# Save summarized data to CSV file
csv_file = './preprocessed/blockchainmedia.csv'
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['Title', 'Date', 'Summary'])
    writer.writeheader()
    for article in all_articles:
        writer.writerow(article)

# Print the contents of the CSV file
print(f"Data has been saved to {csv_file}")

df = pd.read_csv(csv_file)
print(df)

c:\Users\Reyhan\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\Reyhan\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


Data has been saved to ./preprocessed/blockchainmedia.csv
                                                 Title            Date  \
0    Michael Saylor Tetap Optimis Meski Harga BTC M...    23 June 2024   
1    Masa Depan Toncoin: Harga TON di Ambang Kebang...    23 June 2024   
2    Think Tank Korea Selatan Ragukan ETF Bitcoin, ...    24 June 2024   
3    Dogwifhat (WIF) Keluar dari 50 Besar, Harga An...    24 June 2024   
4    XRP Ripple Terus Berkonsolidasi, Kapan Harga A...    24 June 2024   
..                                                 ...             ...   
291  Shiba Inu Hadapi Penurunan, Peluang Akumulasi ...  19 August 2024   
292  Sinyal Bullish Muncul! Bitcoin Siap Kembali ke...  19 August 2024   
293  Lufina Hadirkan Game Berhadiah Kripto FINA Set...  19 August 2024   
294  Tekanan Jual PEPE Semakin Kuat dan Potensi Nai...  19 August 2024   
295  Ekonomi AS Tangguh, Analis Reku: Imbas ke Pasa...  19 August 2024   

                                               Summar